In [39]:
import csv
import cv2
import numpy as np
import torch
from torch.utils.data import Dataset,DataLoader,TensorDataset
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as nnfc
import torch.optim as op

In [40]:
class Mydata(Dataset):
    def __init__(self, data, lab):
        self.data = data
        self.label = lab
    def __getitem__(self, idx):
        data=self.data[idx]
        label=self.label[idx]
        return data, label
    def __len__(self):
        return  len(self.data)

In [41]:
class cnn_net(nn.Module):
    def __init__(self):
        super(cnn_net,self).__init__()
        self.conv1=nn.Sequential(
        nn.Conv2d(
            in_channels = 3,
            out_channels = 16,
            kernel_size = 3,
            stride = 1,
            ),
            nn.ReLU(),
            nn.MaxPool2d(2,2),  
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(16,32,3,1),
            nn.ReLU(),
            nn.MaxPool2d(2,2)
        )
        self.dense = torch.nn.Sequential(
            nn.Linear(397312,512),
            nn.ReLU(),
            nn.Linear(512,88)
        )
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0),-1)
        output = self.dense(x)
        return output

In [42]:
trainFile='train_truth.csv'
trainPath=r'music_train/music_train/'

In [44]:
if __name__=='__main__':
    imgData = []
    imgLabel = []
    with open(trainFile) as csvfile:
        line = csv.reader(csvfile)
        for i in line:
            if i[1] == 'category': continue
            imgData1 = cv2.imread(trainPath + i[0])
            imgData1 = imgData1.astype(np.float32) / 255
            imgData.append(torch.from_numpy(imgData1).reshape(-1,394,520))
            imgLabel.append(np.array(int(i[1])))
    criterion = nn.CrossEntropyLoss()
    trainingDataset = Mydata(imgData,imgLabel)
    trainLoader = DataLoader(dataset=trainingDataset, batch_size=8,shuffle=True)
    net = cnn_net()
    if torch.cuda.is_available():
        net = cnn_net().cuda()
    optimizer = op.SGD(net.parameters(), lr=0.001, momentum=0.9)
    for epoch in range(2):
        running_loss =0.0
        long = len(trainLoader)
        for i, data in enumerate(trainLoader, 0):
            inputs, labels = data
            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, labels.long()) #不用long會報錯
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            if (i+1) % 10 == 0:
                print('Epoch [%d/%d], Iter [%d/%d] Loss: %.6f' %(epoch + 1, 5, i+1,long,loss.item()))
print('Finished Training')

Epoch [1/5], Iter [10/356] Loss: 4.464367
Epoch [1/5], Iter [20/356] Loss: 4.451199
Epoch [1/5], Iter [30/356] Loss: 4.490339
Epoch [1/5], Iter [40/356] Loss: 4.512022
Epoch [1/5], Iter [50/356] Loss: 4.388556
Epoch [1/5], Iter [60/356] Loss: 4.448438
Epoch [1/5], Iter [70/356] Loss: 4.481010
Epoch [1/5], Iter [80/356] Loss: 4.491033
Epoch [1/5], Iter [90/356] Loss: 4.474690
Epoch [1/5], Iter [100/356] Loss: 4.537066
Epoch [1/5], Iter [110/356] Loss: 4.491563
Epoch [1/5], Iter [120/356] Loss: 4.462576
Epoch [1/5], Iter [130/356] Loss: 4.478183
Epoch [1/5], Iter [140/356] Loss: 4.478676
Epoch [1/5], Iter [150/356] Loss: 4.456571
Epoch [1/5], Iter [160/356] Loss: 4.490381
Epoch [1/5], Iter [170/356] Loss: 4.444794
Epoch [1/5], Iter [180/356] Loss: 4.504864
Epoch [1/5], Iter [190/356] Loss: 4.504969
Epoch [1/5], Iter [200/356] Loss: 4.484937
Epoch [1/5], Iter [210/356] Loss: 4.487219
Epoch [1/5], Iter [220/356] Loss: 4.440610
Epoch [1/5], Iter [230/356] Loss: 4.385356
Epoch [1/5], Iter [2

In [47]:
torch.save({'state_dict': net.state_dict()}, 'checkpoint.pth.tar')

In [48]:
Tfilename = []
textPath = r'music_test/music_test/'
import os
import pandas as pd
def testfileList(PATH):
    FileList = os.listdir(PATH)
    return FileList
Tfilename.append(testfileList(textPath))
cfilename = []
for i in Tfilename:
    cfilename = i 
    print(i)
df = pd.DataFrame({'filename': cfilename, 'category':''})
df.to_csv('./testfile.csv', index=False)


['S1m01_00.png', 'S1m01_01.png', 'S1m01_02.png', 'S1m01_03.png', 'S1m01_04.png', 'S1m01_05.png', 'S1m01_06.png', 'S1m01_07.png', 'S1m02_00.png', 'S1m02_01.png', 'S1m02_02.png', 'S1m02_03.png', 'S1m02_04.png', 'S1m02_05.png', 'S1m02_06.png', 'S1m02_07.png', 'S1m03_00.png', 'S1m03_01.png', 'S1m03_02.png', 'S1m03_03.png', 'S1m03_04.png', 'S1m03_05.png', 'S1m03_06.png', 'S1m03_07.png', 'S1m04_00.png', 'S1m04_01.png', 'S1m04_02.png', 'S1m04_03.png', 'S1m04_04.png', 'S1m04_05.png', 'S1m04_06.png', 'S1m04_07.png', 'S1m05_00.png', 'S1m05_01.png', 'S1m05_02.png', 'S1m05_03.png', 'S1m05_04.png', 'S1m05_05.png', 'S1m05_06.png', 'S1m05_07.png', 'S1m06_00.png', 'S1m06_01.png', 'S1m06_02.png', 'S1m06_03.png', 'S1m06_04.png', 'S1m06_05.png', 'S1m06_06.png', 'S1m06_07.png', 'S1m07_00.png', 'S1m07_01.png', 'S1m07_02.png', 'S1m07_03.png', 'S1m07_04.png', 'S1m07_05.png', 'S1m07_06.png', 'S1m07_07.png', 'S1m08_00.png', 'S1m08_01.png', 'S1m08_02.png', 'S1m08_03.png', 'S1m08_04.png', 'S1m08_05.png', 'S1m08_

In [49]:
testFile='testfile.csv'
textimg = []
textfilename = []
with open(testFile) as csvfile:
        line = csv.reader(csvfile)
        for i in line:
            if i[1] == 'category': continue
            imgData1 = cv2.imread(textPath + i[0])
            imgData1 =imgData1.astype(np.float32) / 255
            textimg.append(torch.from_numpy(imgData1).reshape(-1,394,520))
            textfilename.append(i[0])
print(textimg)
print(textfilename)

[tensor([[[1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
         [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
         [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
         ...,
         [0.2431, 0.7922, 0.4471,  ..., 0.6667, 0.4902, 0.2000],
         [0.6667, 0.4902, 0.2000,  ..., 1.0000, 1.0000, 1.0000],
         [1.0000, 1.0000, 1.0000,  ..., 0.2235, 0.7373, 0.4706]],

        [[0.2235, 0.7373, 0.4706,  ..., 0.6196, 0.4980, 0.1843],
         [0.6196, 0.4980, 0.1804,  ..., 1.0000, 1.0000, 1.0000],
         [1.0000, 1.0000, 1.0000,  ..., 0.2235, 0.7373, 0.4706],
         ...,
         [0.6824, 0.4824, 0.2078,  ..., 1.0000, 1.0000, 1.0000],
         [1.0000, 1.0000, 1.0000,  ..., 0.2353, 0.7725, 0.4549],
         [0.2353, 0.7725, 0.4549,  ..., 0.6824, 0.4824, 0.2039]],

        [[0.6824, 0.4824, 0.2078,  ..., 1.0000, 1.0000, 1.0000],
         [1.0000, 1.0000, 1.0000,  ..., 0.2353, 0.7725, 0.4549],
         [0.2353, 0.7725, 0.4549,  ..., 0.6824, 0.4824, 0

         [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000]]])]
['S1m01_00.png', 'S1m01_01.png', 'S1m01_02.png', 'S1m01_03.png', 'S1m01_04.png', 'S1m01_05.png', 'S1m01_06.png', 'S1m01_07.png', 'S1m02_00.png', 'S1m02_01.png', 'S1m02_02.png', 'S1m02_03.png', 'S1m02_04.png', 'S1m02_05.png', 'S1m02_06.png', 'S1m02_07.png', 'S1m03_00.png', 'S1m03_01.png', 'S1m03_02.png', 'S1m03_03.png', 'S1m03_04.png', 'S1m03_05.png', 'S1m03_06.png', 'S1m03_07.png', 'S1m04_00.png', 'S1m04_01.png', 'S1m04_02.png', 'S1m04_03.png', 'S1m04_04.png', 'S1m04_05.png', 'S1m04_06.png', 'S1m04_07.png', 'S1m05_00.png', 'S1m05_01.png', 'S1m05_02.png', 'S1m05_03.png', 'S1m05_04.png', 'S1m05_05.png', 'S1m05_06.png', 'S1m05_07.png', 'S1m06_00.png', 'S1m06_01.png', 'S1m06_02.png', 'S1m06_03.png', 'S1m06_04.png', 'S1m06_05.png', 'S1m06_06.png', 'S1m06_07.png', 'S1m07_00.png', 'S1m07_01.png', 'S1m07_02.png', 'S1m07_03.png', 'S1m07_04.png', 'S1m07_05.png', 'S1m07_06.png', 'S1m07_07.png', 'S1m08_00.png', 'S1m08_01.png', 'S1

In [50]:
testingDataset = Mydata(textimg,textfilename)
testLoader = DataLoader(dataset=testingDataset, batch_size=8)
print(testLoader)
final = []
net = cnn_net()
if torch.cuda.is_available():
    net = cnn_net().cuda()
checkpoint = torch.load('checkpoint.pth.tar')
net.load_state_dict(checkpoint['state_dict'])
net.eval()
with torch.no_grad():
    for data in testLoader:
        inputs, _ = data
        if torch.cuda.is_available():
            prediction = net(inputs.cuda())
        else:
            prediction = net(inputs)
        predictionLabel = torch.argmax(prediction, dim=1)
        final.append(predictionLabel.cpu())
final=torch.cat(final).int()
df = pd.DataFrame({'filename': textfilename, 'category': final})
df.to_csv('./pred.csv', index=False)

In [ ]:
## 參考資料:
#http://moocs.nccu.edu.tw/course/172/intro
#https://morvanzhou.github.io/tutorials/machine-learning/torch/
#https://www.youtube.com/watch?v=Hos_xO0nKjs
#https://github.com/MorvanZhou/PyTorch-Tutorial/blob/master/tutorial-contents/401_CNN.py
#https://blog.gtwang.org/programming/opencv-basic-image-read-and-write-tutorial/
#https://www.youtube.com/watch?v=GbHZYbWRBqo
#https://medium.com/codeda/pytorch-%E5%AD%B8%E7%BF%92-cnn-day8-20-20f636a3ffdc
#https://zhuanlan.zhihu.com/p/67058668
#https://blog.csdn.net/fendouaini/article/details/102938089
#https://blog.csdn.net/wblgers1234/article/details/75330923?utm_medium=distribute.pc_relevant.none-task-blog-BlogCommendFromMachineLearnPai2-3.nonecase&depth_1-utm_source=distribute.pc_relevant.none-task-blog-BlogCommendFromMachineLearnPai2-3.nonecase
#https://zhuanlan.zhihu.com/p/30117574